In [1]:
from utils.models import random_forest, log_reg
from utils.plot import evaluation_metrics
from utils.model import get_cleaned_data, split_train_test, finetune_models
data = get_cleaned_data("./data/kickstarter_projects.csv")
xtrain,xtest,ytrain,ytest = split_train_test(data, "success")


In [2]:
random_forest = random_forest.fit(xtrain, ytrain)
log_reg = log_reg.fit(xtrain,ytrain)

In [5]:

evaluation_metrics([("log_reg",log_reg), ("random_forest",random_forest)],xtest,ytest)


model
model
   Accuracy Score  Precision Score  Recall Score  F1 Score
0            0.90             0.92          0.80      0.86
1            0.75             0.93          0.34      0.50
